In [3]:
import openpyxl
import pyomo.environ as pe
import numpy as np
import pandas as pd
from pyomo.core import expr


In [4]:
#################################################################################################################
# Objective: Read some appliance's characteristics from the excel "Info".
#
# ->Dev: Set of devices (1 a 15);
# ->Type: 0 (Not Flexibel), 1 (Power Flexibility), 2 (Time Flexibility) and 3 (Time and Power Flexibility);
# ->T_ON: Time that each load needs to be turned ON;
# ->T_OFF: Time that each load needs to be turned ON and OFF;
# ->State_init: Initial state (t=0) of each appliance: 0 (desligada) ou 1 (ligada);
# ->P_max: Maximum Power for each appliance.
# ->W_max: Maximum Weight for each appliance;
# ->W_min: Minimum Weight for each appliance;
# ->Y_max: Maximum Power percentage for each appliance;
# ->Y_min: Minimum Power percentage for each appliance;
# ->T_Hold_Power: Time needed to fix a power change for each appliance.
#
#################################################################################################################

# load the workbook
workbook = openpyxl.load_workbook('Info.xlsx')

# select the worksheet
worksheet = workbook['Lds_inputs']

# read 'Set of devices'
column = worksheet['B']
Dev = []
for cell in column:
    if cell.row == 1:
        continue
    Dev.append(cell.value)

# read 'Type of appliances'
column = worksheet['C']
Type = []
for cell in column:
    if cell.row == 1:
        continue
    Type.append(cell.value)

# read 'Time needed to be ON' 
column = worksheet['D']
T_ON = []
for cell in column:
    if cell.row == 1:
        continue
    T_ON.append(cell.value)

# read 'Time needed to be OFF' 
column = worksheet['E']
T_OFF = []
for cell in column:
    if cell.row == 1:
        continue
    T_OFF.append(cell.value)

# read 'Initial State'
column = worksheet['F']
State_init = []
for cell in column:
    if cell.row == 1:
        continue
    State_init.append(cell.value)

# read 'Maximum Power'
column = worksheet['G']
P_max = []
for cell in column:
    if cell.row == 1:
        continue
    P_max.append(cell.value)

# read 'Maximum Weight'
column = worksheet['H']
W_max = []
for cell in column:
    if cell.row == 1:
        continue
    W_max.append(cell.value)

# read 'Minimum Weight'
column = worksheet['I']
W_min = []
for cell in column:
    if cell.row == 1:
        continue
    W_min.append(cell.value)

# read 'Maximum percentage of power'
column = worksheet['J']
Y_max = []
for cell in column:
    if cell.row == 1:
        continue
    Y_max.append(cell.value)

# read 'Minimum percentage of power'
column = worksheet['K']
Y_min = []
for cell in column:
    if cell.row == 1:
        continue
    Y_min.append(cell.value)

for j in range(len(Dev)):
    Y_max [j] = Y_max [j] / 100
    Y_min [j] = Y_min [j] / 100

# read 'Time needed to fix a power change'
column = worksheet['L']
T_Hold_Power = []
for cell in column:
    if cell.row == 1:
        continue
    T_Hold_Power.append(cell.value)

# close the workbook
workbook.close()

# Print the values
print(f"Index of devices - {(Dev)}")
print(f"Type of appliances - {(Type)}")
print(f"T_ON - {(T_ON)}")
print(f"T_OFF - {(T_OFF)}")
print(f"Initial State - {(State_init)}")
print(f"Maximum Power - {(P_max)}")
print(f"Maximum Weight - {(W_max)}")
print(f"Minimum Weight - {(W_min)}")
print(f"Y_max - {(Y_max)}")
print(f"Y_min - {(Y_min)}")
print(f"T_Hold_Power - {(T_Hold_Power)}")

Index of devices - [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Type of appliances - [2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0]
T_ON - [6, 15, 7, 10, 38, 32, 50, 40, 40, 40, 3, 10]
T_OFF - [4, 4, 45, 40, 65, 60, -1, -1, -1, -1, -1, -1]
Initial State - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Maximum Power - [590, 500, 100, 120, 100, 2000, 138, 72, 40, 30, 1000, 300]
Maximum Weight - [10, 10, 10, 10, 10, 10, 6, 4, 3, 5, 10, 10]
Minimum Weight - [10, 10, 10, 10, 10, 10, 4, 1, 2, 3, 10, 10]
Y_max - [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Y_min - [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.8, 0.8, 0.8, 1.0, 1.0]
T_Hold_Power - [-1, -1, -1, -1, -1, -1, 3, 10, 10, 10, -1, -1]


In [5]:
#################################################################################################################
# Objective: Declaration of sets, parameters and variables.
#
# ->Sets: T, Dev
# ->Parameters: P_lim
# ->Mutable Parameters: 
# ->Variables:
#
#################################################################################################################


# Sets and Constants:
T=20 # Set of indices in scheduling horizon        
P_lim = 5000 # Power limit   
P_rlx = [0] * T
rows = T # Auxiliary variable
cols = len(Dev) # Auxiliary variable

# On/Off variables:
X_onoff_FC_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
X_opt_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i

# Power variables:
P_sum=[0] * T # Sum of Power of all appliances at time i
Y_onoff_FC_matrix = [[1 for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
Y_matrix = [[1 for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
Y_opt_matrix = [[1 for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
P_aux = [0] * T

# Weight variables:
W_FC_matrix = [[10 for j in range(cols)] for i in range(rows)] # Weight of appliance j at time i
WT_opt_matrix = [[10 for j in range(cols)] for i in range(rows)] # Weight of appliance j at time i

# Auxiliary variables:
cont_on_dataset = [0] * len(Dev)
cont_off_dataset = [0] * len(Dev)
state = [0] * len(Dev) #(0=desligado, 1=ligado)
T_ON_atualizado = [0] * len(Dev)
T_ON_atualizado = T_ON.copy()


Power_aux =[[0 for j in range(cols)] for i in range(rows)]
aux =[[0 for j in range(cols)] for i in range(rows)]
P =[[0 for j in range(cols)] for i in range(rows)]
P_sum_aux = [0] * T

P_opt= [[0 for j in range(cols)] for i in range(rows)]
P_sum_opt = [0] * T
differences = [0] * T

X_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
W_Time_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
W_Time_opt_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
X_opt_matrix_real = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i

In [6]:
# Cria matriz de 0 e 1.
# Neste caso, nao depende do peso, este dataset é como se fosse um dado de entrada que me dao

def function_dataset ():

    """ Before Optimization: """
    for i in range(T):
        for j in range(len(Dev)): 

            # StaFC ON 
            if (i==0 and State_init[j] == 1):     
                X_onoff_FC_matrix[i][j] = 1
                cont_on_dataset[j] += 1
                state[j]= 1
            
            # StaFC OFF
            elif (i==0 and State_init[j] == 0):         
                X_onoff_FC_matrix[i][j] = 0
                cont_off_dataset[j] += 1
                state[j]= 0
        
            # Loads with time flexibility
            # Loads with power flexibility 
            # Loads non-flexible
            if (i!=0):

                if state[j] == 1 and (Type[j]== 2 or Type[j]== 3):                              
                    if cont_on_dataset[j] < T_ON_atualizado[j]:                    
                        X_onoff_FC_matrix[i][j] = 1
                        cont_on_dataset[j] += 1
                        state[j]=1
                    else:                                       
                        cont_on_dataset[j] = 0
                        X_onoff_FC_matrix[i][j] = 0
                        cont_off_dataset[j] = 1
                        state[j]=0

                elif state[j] == 0 and (Type[j]== 2 or Type[j]== 3):                              
                    if cont_off_dataset[j] < T_OFF[j]:                    
                        X_onoff_FC_matrix[i][j] = 0
                        cont_off_dataset[j] += 1
                        state[j]=0
                    else:                                       
                        cont_on_dataset[j] = 1
                        X_onoff_FC_matrix[i][j] = 1
                        cont_off_dataset[j] = 0
                        state[j]=1

                elif state[j] == 0 and (Type[j]== 1 or Type[j]== 0):                          
                    if ((Type[j]== 1) and (i==1*60 or i==13*60 or i==19*60 or i==20*60 or i==21*60)) or ((Type[j]== 0) and (i==12*60 or i==23*60 or i==20*60)): 
                    #Ligar cargas com flex. em potencia (luzes) às 07:00 , 19:00 , 20:00 e 21:00; 
                    #Ligar cargas não flexiveis as 13:00 e 20:00;
                        X_onoff_FC_matrix[i][j] = 1
                        cont_on_dataset[j] += 1
                        state[j] = 1
                    else:
                        X_onoff_FC_matrix[i][j] = 0
                        state[j] = 0
                        cont_on_dataset[j] = 0

                elif state[j] == 1 and (Type[j]== 1 or Type[j]== 0): 
                    if cont_on_dataset[j] < T_ON_atualizado[j]:                    
                        cont_on_dataset[j] += 1
                        X_onoff_FC_matrix[i][j] = 1
                        state[j] = 1
                    else:
                        X_onoff_FC_matrix[i][j] = 0
                        state[j] = 0
                        cont_on_dataset[j] = 0
                       
    return X_onoff_FC_matrix


In [7]:
#######################################################################################################################
# Dá um peso comnsoante a matriz de dataset de entrada
#Objective: Provide weights relatively to the time component for each load to represent the comfoFC level of the user.
#
#           1) Weights for each load before optimization;
#           2) Weights for each load after optimization;
# 
#           -> If the load is ON, the weight decreases from 10 to 1 (priority decreases, the load can be turned OFF);  
#           -> If the load is OFF, the weight increases from 1 to 10 (priority increases, the load must be turned ON);       
#           
#           Note: Since we are working in real time, we only want to cut loads that are ON with priorities close to 1,
#           so the weights calculated when the loads are OFF are not relevant.
#       
# Output: Vector with the time component of weights for each load.    
#######################################################################################################################

def function_weight (X_onoff_FC_matrix):
   
    # Before Optimization: 
    for i in range (T):

        for j in range(len(Dev)): 

            # Loads with time flexibility
            if ((Type[j]==2 or Type[j]==3)):
                if (i==0 and State_init[j] == 1): # StaFC ON   
                    W_FC_matrix[i][j] = 10 - (9/T_ON_atualizado[j])   # Weight
                    
                elif (i==0 and State_init[j] == 0): # StaFC OFF
                    W_FC_matrix[i][j] = 1 + (9/T_OFF[j])              # Weight

            # Loads non-flexible
            # Loads with power flexibility
            if (i==0 and (Type[j] == 0 or Type[j]) ) == 1:
                W_FC_matrix[i][j] = 10                                # Weight


            # Loads with time flexibility
            if i!= 0 and (Type[j]==2 or Type[j]==3):
                if X_onoff_FC_matrix[i][j] == 1 : # State ON                                                                                               
                    W_FC_matrix [i][j] =  W_FC_matrix [i-1][j] - (9/T_ON_atualizado[j] ) # Weight decreases
                if X_onoff_FC_matrix[i][j] == 0 : # State OFF 
                     W_FC_matrix [i][j] =  W_FC_matrix [i-1][j] + (9/T_OFF[j] ) # Weight increases
        
            W_FC_matrix [i][j]=( W_FC_matrix [i][j])


    return (W_FC_matrix)



In [8]:
#######################################################################################################################
# Objective: Sum the power for each load at time i.
#
#           1) Before optimization, X= X before optimization;
#           2) After optimization, X= X optimized;
#       
# Output: Value with the sum of power at time i.    
#######################################################################################################################

def function_Power (Y, X):
    
    # Initialize variables to zero
    for i in range(T):
        for j in range(len(Dev)):  
            Power_aux [i][j] = 0
            

    # Power of each device 
    for i in range(T):
        for j in range(len(Dev)):
            Power_aux [i][j] = P_max [j] * Y[i][j] * X[i][j]

    return(Power_aux)

In [9]:
#######################################################################################################################
# Objective: Sum the power for each load at time i.
#
#           1) Before optimization, X= X before optimization;
#           2) After optimization, X= X optimized;
#       
# Output: Value with the sum of power at time i.    
#######################################################################################################################

def function_Power_Sum (Power_aux):
    
    # Initialize variables to zero
    for i in range(T):
        P_sum_aux [i]= 0

    # Sum of device's power at each time i
    for i in range(T):
        for j in range(len(Dev)):  
            P_sum_aux[i] = P_sum_aux [i] + Power_aux[i][j]

    return(P_sum_aux)

In [10]:
X_onoff_FC_matrix = function_dataset()
W_FC_matrix = function_weight(X_onoff_FC_matrix)  
P = function_Power(Y_onoff_FC_matrix, X_onoff_FC_matrix) 
P_sum = function_Power_Sum(P) 

In [11]:

######################################################################################################################
# Optimization: Maximize the user's comfort.
#
# Constraint 1: The sum of each load's power has to be lower than the threshold;
# Constraint 2: The power's percentage that can be decreased has to be higher than the minimum;
# Constraint 3: The power's percentage that can be decreased has to be lower than the maximum;
# Constraint 4: The status of loads with power flexibility, non-flexible, or weight=-1 can not be changed; 
#               The status of loads with time flexibility and weight!=-1 can be changed: 
#                   -If it is on, it can keep or turn off;
#                   -If it is off, it just can keep turned off;
#
# Constraint 5: If the power is not the same as last time i and the load is turned on, the optimization must keep it.
#
# Outputs:  - Maximum comfort; 
#           - Vetor of load's status, X_opt; 
#           - Vetor with load's power percentage, Y_opt;
#           - The power that exceeds the limit, P_rlx.
######################################################################################################################

model = pe.ConcreteModel()

# Set Declaration
model.loads = pe.Set(initialize=np.arange(len(Dev)))
model.Time = pe.Set(initialize=np.arange(T))

X_opt_initial_values = [[0 for j in model.loads] for i in model.Time]
Y_opt_initial_values = [[1 for j in model.loads] for i in model.Time]
W_time_opt_initial_values = [[1 for j in model.loads] for i in model.Time]
VR_initial_values = [[0 for j in model.loads] for i in model.Time]

# Parameter Declaration
model.P_lim = pe.Param(initialize=P_lim)
model.Type = pe.Param(model.loads, initialize=Type)
model.P_max = pe.Param(model.loads, initialize=P_max)
model.State_init = pe.Param(model.loads, initialize=State_init)
model.T_OFF = pe.Param(model.loads, initialize=T_OFF)
model.T_ON = pe.Param(model.loads, initialize=T_ON)
model.W_max = pe.Param(model.loads, initialize=W_max)
model.W_min = pe.Param(model.loads, initialize=W_min)
model.Y_max = pe.Param(model.loads, initialize=Y_max)
model.Y_min = pe.Param(model.loads, initialize=Y_min)
model.X = pe.Param(model.Time, model.loads, initialize=lambda m, i, j: X_onoff_FC_matrix[i][j])
model.W_time = pe.Param(model.Time, model.loads, initialize=lambda m, i, j: W_FC_matrix[i][j])
model.Y = pe.Param(model.Time, model.loads, initialize=lambda m, i, j: Y_onoff_FC_matrix[i][j])


# Variable Declaration
model.X_opt = pe.Var(model.Time, model.loads, domain=pe.Binary, initialize=lambda m, i, j: X_opt_initial_values[i][j])
model.W_time_opt = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j:W_time_opt_initial_values [i][j])
model.P_rlx = pe.Var(model.Time, domain=pe.NonNegativeReals, initialize=P_rlx)
#model.W_power_opt_aux = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j:W_time_opt_initial_values [i][j])
model.W_power_opt = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j:W_time_opt_initial_values [i][j])
model.Y_opt = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j: Y_opt_initial_values[i][j])
model.comfort = pe.Var(model.Time, domain=pe.NonNegativeReals, initialize=differences)


# Constrait 1
def constraint_one(m, i):
    return sum((m.P_max[j] * m.X_opt[i, j]*m.Y_opt[i,j] ) for j in m.loads) <= m.P_lim + m.P_rlx[i]
model.constraint_one = pe.Constraint(model.Time, rule=constraint_one)


# Constrait 2
def _constraint_two(m, i, j):
    if m.Type[j] == 0 or m.Type[j] == 1:
        return m.X_opt[i, j] == m.X[i, j]  
    else:
        return pe.Constraint.Feasible
model.constraint_two = pe.Constraint( model.Time, model.loads, rule=_constraint_two)


# Constrait 3
def _constraint_three(m, i, j):
    if m.Type[j] == 2 or m.Type[j] == 3:
        return m.W_time_opt[i,j] <= 10 
    else:   
        return pe.Constraint.Feasible
model.constraint_three = pe.Constraint(model.Time, model.loads, rule=_constraint_three)


# Constrait 4
def _constraint_four(m, i, j):
    if m.Type[j] == 2 or m.Type[j] == 3:
        return 1  <= m.W_time_opt[i,j] 
    else:   
        return pe.Constraint.Feasible
model.constraint_four = pe.Constraint(model.Time, model.loads, rule=_constraint_four)


# Constrait 5
def constraint_five(m, i, j):
    if m.Type[j] == 0 or m.Type[j] == 1:
        return m.W_time_opt[i, j] == 10
    elif m.Type[j] == 2 or m.Type[j] == 3:
        if i==0:
            return m.W_time_opt[i, j] == ((1 + 9/m.T_OFF[j]) * (1 - m.X_opt[i, j]) + (10 - 9/m.T_ON[j]) * m.X_opt[i, j] )
        elif i!= 0:
            return m.W_time_opt[i, j] ==( (m.W_time_opt[i-1, j] - 9/m.T_ON[j]) * m.X_opt[i, j] + (m.W_time_opt[i-1, j] + 9/m.T_OFF[j]) * (1 - m.X_opt[i, j]) ) 
        else:   
            return pe.Constraint.Skip
    else:   
        return pe.Constraint.Skip
model.constraint_five = pe.Constraint(model.Time, model.loads, rule=constraint_five)


# Constrait 6
def _constraint_six(m, i, j):
        return ((m.Y_min[j] <= m.Y_opt[i,j]))
model.constraint_six = pe.Constraint(model.Time, model.loads, rule=_constraint_six)

# Constrait 7
def _constraint_seven(m, i, j):
    return ((m.Y_opt[i,j] <= m.Y_max[j]))
model.constraint_seven = pe.Constraint(model.Time, model.loads, rule=_constraint_seven)


# Constrait 8
def _constraint_eight(m, i, j):
    if m.Type[j] == 0 or m.Type[j] == 2:
        return m.Y_opt[i, j] == m.Y[i, j]  
    else:
        return pe.Constraint.Feasible
model.constraint_eight = pe.Constraint( model.Time, model.loads, rule=_constraint_eight)



def _constraint_OF(m, i):
    return m.comfort[i] == sum(m.W_time_opt[i,j] + m.Y_opt[i,j] for j in range(len(Dev)))
model.constraint_OF = pe.Constraint(model.Time, rule=_constraint_OF)
"""

def constraint_a_rule(m, i, j):
    return m.W_power_opt[i, j] == ( m.W_min[j] +  ((m.W_max[j] - m.W_min[j]) / (m.Y_max[j] - m.Y_min[j] + 0.000001))*(m.Y_opt[i,j] - m.Y_min[j]) ) * m.X_opt[i, j]
model.constraint_a = pe.Constraint(model.Time, model.loads, rule=constraint_a_rule)

def _constraint_OF(m, i):
    return m.comfort[i] == sum(m.W_time_opt[i,j] + m.W_power_opt[i,j] for j in range(len(Dev)))
model.constraint_OF = pe.Constraint(model.Time, rule=_constraint_OF)
"""

def _objective_function(m):
    return sum(m.comfort[i] - (1000*m.P_rlx[i])  for i in m.Time)
model.objective = pe.Objective(expr=_objective_function, sense=pe.maximize)





In [12]:
#Resolução do problema FC
                             
        
print("\n--------------------------------Before Optimization----------------------------------------\n")

print("\nForecast X: ")       
for i in range(T):
    print(X_onoff_FC_matrix [i])

print("\nW_Time: ")
for i in range(T):
    print(W_FC_matrix [i])

print("\nPower: ")
for i in range(T):
    print(f"i=[{i+1}] = {P[i]}")

print("\nY: ")
for i in range(T):
    print(f"i=[{i+1}] = {Y_onoff_FC_matrix[i]}")

print("\nP_sum: ")
for i in range(T):
    print(f"i=[{i+1}] = {P_sum[i]}")
       
# Solver                                                 
results = pe.SolverFactory('scip', executable='C:/Program Files/SCIPOptSuite 8.0.3/bin/scip.exe').solve(model)
#model.pprint()


print("\n-----------------------------Model Results--------------------------------------------\n")

for i in model.Time:
    for j in range(len(Dev)):
        X_matrix [i][j] = model.X[i,j]
        W_Time_matrix [i][j] = round(model.W_time[i,j],2)
        X_opt_matrix [i][j] = round(model.X_opt[i,j].value) # Status on/off of each loads after optimization
        W_Time_opt_matrix [i][j] = round(model.W_time_opt[i,j].value,2)
        Y_matrix[i][j] = model.Y[i,j] * model.X[i,j]
        Y_opt_matrix[i][j] = round(model.Y_opt[i,j].value,2) * round(model.X_opt[i,j].value,2)


print("\nX: ")
for i in range(T):
    print(X_matrix[i])

print("\nY: ")
for i in range(T):
    print(Y_matrix [i])

print("\nW_Time: ")
for i in range(T):
    print(W_Time_matrix[i])

print("\nX_opt: ")
for i in range(T):
    print(X_opt_matrix [i])

print("\nW_Time_opt: ")
for i in range(T):
    print(W_Time_opt_matrix [i])

print("\nY_opt: ")
for i in range(T):
    print(Y_opt_matrix [i])



OF = model.objective()
print('\nMin differences  =', OF)



# Initialize variables to zero
for i in range(T):
    for j in range(len(Dev)):  
        P_opt [i][j] = 0
        

# Power of each device 
for i in range(T):
    for j in range(len(Dev)):
        P_opt [i][j] = P_max [j] * model.Y_opt[i,j].value * model.X_opt[i,j].value


for i in range(T):
    P_sum_opt [i]= 0

# Sum of device's power at each time i
for i in range(T):
    for j in range(len(Dev)):  
        P_sum_opt[i] = P_sum_opt [i] + P_opt[i][j]

#P_sum_opt = function_Power_Sum(P_opt) 

for i in range(T):
    for j in range(len(Dev)):
        P [i][j] = round(P [i][j],2)
        P_opt [i][j] = round(P_opt [i][j],2)
    P_sum [i] = round(P_sum [i],2)
    P_sum_opt [i] = round(P_sum_opt [i],2)


print("\nPower: ")
for i in range(T):
    print(f"i=[{i+1}] = {P[i]}")

print("\nP_sum: ")
for i in range(T):
    print(f"i=[{i+1}] = {P_sum[i]}")

print("\nPower Opt: ")
for i in range(T):
    print(f"i=[{i+1}] = {P_opt[i]}")

print("\nP_sum Opt: ")
for i in range(T):
    print(f"i=[{i+1}] = {P_sum_opt[i]}")


for i in range(T):
    P_rlx [i]= round(model.P_rlx[i].value,2)
print("\nP_rlx: ")
for i in range(T):
    print(f"i=[{i+1}] = {P_rlx[i]}")





--------------------------------Before Optimization----------------------------------------


Forecast X: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]
[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]

W_Time: 
[3.25, 3.25, 1.2, 1.225, 1.1384615384615384, 1.15, 10, 10, 10, 10, 10, 10]
[5.5, 5.5, 1.4, 1.4500000000000002, 1.2769230769230768, 1.299999999


-----------------------------Model Results--------------------------------------------


X: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]
[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]

Y: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 1, 0, 0, 

In [13]:
#################################################################################################################
# Objective: ->Add labels for a better understanding of the results.
#            ->Add the minutes and indexes to the matrix of the status on/off and weights.
#            ->Add the sum of load's power before and after the optimization to the matrix of status on/off.
#################################################################################################################


import datetime

col = []
for i in range(T):
    # Crie um objeto datetime representando o tempo
    time_value = datetime.datetime(1, 1, 1, i // 60, i % 60)

    # Formate o objeto datetime como uma string no formato desejado (hora:minuto:segundo)
    time_str = time_value.strftime("%H:%M:%S")

    col.append(time_str)

for i in range(T):
    X_matrix [i].insert(0, col[i])
    W_Time_matrix [i].insert(0, col[i])

    X_opt_matrix [i].insert(0, col[i])
    W_Time_opt_matrix[i].insert(0, col[i])

    P [i].insert(0, col[i])
    P_opt [i].insert(0, col[i])

    Y_matrix [i].insert(0, col[i])
    Y_opt_matrix [i].insert(0, col[i])



for i in range(T):
    X_matrix[i].append(0)
    X_matrix[i].append(0)
    X_matrix[i].append(0)

    X_opt_matrix[i].append(0)
    X_opt_matrix[i].append(0)
    X_opt_matrix[i].append(0)

    
    P[i].append(0)
    P[i].append(0)
    P[i].append(0)
    
    P_opt[i].append(0)
    P_opt[i].append(0)
    P_opt[i].append(0)

    Y_matrix[i].append(0)
    Y_matrix[i].append(0)
    Y_matrix[i].append(0)

    Y_opt_matrix[i].append(0)
    Y_opt_matrix[i].append(0)
    Y_opt_matrix[i].append(0)
    

    #change value in ON_or_OFF list in position 16 for sum[i]
    X_matrix[i][13] = P_sum[i]
    X_matrix[i][14] = P_sum_opt[i]
    X_matrix[i][15] = P_rlx [i]

    X_opt_matrix[i][13] = P_sum[i]
    X_opt_matrix[i][14] = P_sum_opt[i]
    X_opt_matrix[i][15] = P_rlx [i]
    
    P[i][13] = P_sum[i]
    P[i][14] = P_sum_opt[i]
    P[i][15] = P_rlx [i]
    
    P_opt[i][13] = P_sum[i]
    P_opt[i][14] = P_sum_opt[i]
    P_opt[i][15] = P_rlx [i]

    Y_matrix[i][13] = P_sum[i]
    Y_matrix[i][14] = P_sum_opt[i]
    Y_matrix[i][15] = P_rlx [i]

    Y_opt_matrix[i][13] = P_sum[i]
    Y_opt_matrix[i][14] = P_sum_opt[i]
    Y_opt_matrix[i][15] = P_rlx [i]
    

row = []
for i in range(len(Dev)+4):
    row.append(i)
row [0] = 'Min/Index'
row [13] ='Sum Power'
row [14] ='Power Optim'
row [15] ='Power Rlx'

row1 = []
for i in range(len(Dev)+1):
    row1.append(i)
row1 [0] = 'Min/Index'

X_matrix.insert(0, row)
W_Time_matrix.insert(0, row1)
X_opt_matrix.insert(0, row)
W_Time_opt_matrix.insert(0, row1)
P.insert(0, row)
P_opt.insert(0, row)
Y_matrix.insert(0, row)
Y_opt_matrix.insert(0, row)



In [14]:
############################################################################################################
# Objectivo: Imprime as 4 worksheets.
# Função: Escreve no excel: 
#               ->tabela com ON/OFF optimizado para cada load para ti;
#               ->tabela com ON/OFF antes da optimização;
#               ->tabela com weight_auxs optimizados;
#               ->tabela com weight_auxs antes da optimização;
############################################################################################################


# Create a Pandas Excel writer using XlsxWriter
writer = pd.ExcelWriter('Results.xlsx', engine='xlsxwriter')

df = pd.DataFrame(X_matrix)
writer = pd.ExcelWriter('Results.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='X before Opt', index=False, header=False)

df2 = pd.DataFrame(W_Time_matrix)
df2.to_excel(writer, sheet_name='W_Time before Opt', index=False, header=False)

df3 = pd.DataFrame(X_opt_matrix)
df3.to_excel(writer, sheet_name='X Opt', index=False, header=False)

df4 = pd.DataFrame(W_Time_opt_matrix)
df4.to_excel(writer, sheet_name='W_Time opt', index=False, header=False)

df5 = pd.DataFrame(P)
df5.to_excel(writer, sheet_name='P before Opt', index=False, header=False)

df6 = pd.DataFrame(P_opt)
df6.to_excel(writer, sheet_name='P Opt', index=False, header=False)

df7 = pd.DataFrame(Y_matrix)
df7.to_excel(writer, sheet_name='Y before Opt', index=False, header=False)

df8 = pd.DataFrame(Y_opt_matrix)
df8.to_excel(writer, sheet_name='Y Opt', index=False, header=False)

writer.save()
writer.close()



C:\Users\anaol\AppData\Local\Temp\ipykernel_17792\2324610636.py:39: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\Users\anaol\AppData\Local\Programs\Python\Python311\Lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
